# 03: Entrenamiento del Modelo para Ayatsil (v2)

* **Autor:** Emmanuel Eduardo
* **Fecha de Versión:** 2025-07-21
* **Versión:** 2.0 (Avanzada)
* **Proyecto:** Pronóstico de Producción - CNH

---

## Objetivo
Este notebook carga los datos con **características avanzadas** de Ayatsil, entrena un modelo XGBoost mejorado (v2), lo evalúa comparándolo con la v1 y guarda los nuevos artefactos.

## Entradas
* `gold/ayatsil_features_advanced_v2.parquet`

## Salidas
* `ml/modelos/ayatsil_xgb_advanced_v2.json`
* `ml/evaluacion/eval_ayatsil_advanced_v2.json`

In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import json
import os


In [6]:
# 0. Definir rutas del proyecto
ruta_base = os.path.abspath(os.path.join(os.getcwd(), "..",".."))
ruta_gold = os.path.join(ruta_base, "gold")
ruta_modelos = os.path.join(ruta_base, "ml", "modelos")
ruta_evaluacion = os.path.join(ruta_base, "ml", "evaluacion")

In [8]:
# 1. Cargar el dataset con características AVANZADAS
# --- CAMBIO: Se carga el nuevo archivo v2 ---
df_modelo = pd.read_parquet(os.path.join(ruta_gold, "ayatsil_features_advanced_v2.parquet"))


In [10]:
# 2. Definir Características (X) y Objetivo (y) - VERSIÓN AVANZADA
# --- CAMBIO: Se incluye la lista completa de features ---
features = [
        'produccion_mes_anterior',
        'produccion_promedio_3m',
        'mes',
        'trimestre',
        'produccion_lag_6',
        'produccion_lag_12',
        'produccion_roll_mean_6',
        'produccion_roll_std_6',
        'año'
    ]
target = 'petroleo_mbd'

X = df_modelo[features]
y = df_modelo[target]
print("\nCaracterísticas (X) y objetivo (y) definidos para Ayatsil (versión avanzada).")


Características (X) y objetivo (y) definidos para Ayatsil (versión avanzada).


In [12]:
# 3. Dividir los datos en Entrenamiento y Prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 4. Entrenar el Modelo XGBoost v2
model_v2 = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        early_stopping_rounds=5
    )
    
model_v2.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )
print("\nModelo para Ayatsil (versión avanzada v2) entrenado con éxito.")


Modelo para Ayatsil (versión avanzada v2) entrenado con éxito.


In [14]:
# 5. Evaluar el Modelo y Mostrar Resultados Comparativos
y_pred = model_v2.predict(X_test)
rmse_v2 = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_v1 = 3.89 # El resultado de nuestro modelo anterior

print("\n--- ¡RESULTADO DEL MODELO MEJORADO V2: AYATSIL! ---")
print(f"RMSE Anterior (v1): {rmse_v1} mbd")
print(f"RMSE Nuevo    (v2): {rmse_v2:.2f} mbd")
print("--------------------------------------------------")

if rmse_v2 < rmse_v1:
    print("¡Éxito! El error se redujo. Las nuevas características mejoraron el modelo.")
else:
    print("El error no se redujo. Es un resultado de aprendizaje valioso.")



--- ¡RESULTADO DEL MODELO MEJORADO V2: AYATSIL! ---
RMSE Anterior (v1): 3.89 mbd
RMSE Nuevo    (v2): 3.46 mbd
--------------------------------------------------
¡Éxito! El error se redujo. Las nuevas características mejoraron el modelo.


In [16]:
# 6. Guardar el Modelo Entrenado v2
os.makedirs(ruta_modelos, exist_ok=True)
# --- CAMBIO: Nuevo nombre de archivo para el modelo v2 ---
ruta_modelo_salida = os.path.join(ruta_modelos, "ayatsil_xgb_advanced_v2.json")
model_v2.save_model(ruta_modelo_salida)
print(f"\nModelo v2 guardado en: {ruta_modelo_salida}")


Modelo v2 guardado en: C:\Users\elezr\Datalquemy\Proyectos\CNH\ml\modelos\ayatsil_xgb_advanced_v2.json


In [18]:
# 7. Guardar la Evaluación del Modelo v2
os.makedirs(ruta_evaluacion, exist_ok=True)
# --- CAMBIO: Nuevo nombre de archivo para la evaluación v2 ---
ruta_evaluacion_salida = os.path.join(ruta_evaluacion, "eval_ayatsil_advanced_v2.json")

info_evaluacion = {
    "nombre_modelo": "ayatsil_xgb_advanced_v2",
    "fecha_entrenamiento": "2025-07-21",
    "metrica": {
        "tipo": "RMSE",
        "valor": round(rmse_v2, 2)
    },
    "comparativa_v1": {
        "rmse_v1": rmse_v1
    }
}

with open(ruta_evaluacion_salida, 'w') as f:
    json.dump(info_evaluacion, f, indent=4)

print(f"Evaluación v2 guardada en: {ruta_evaluacion_salida}")

Evaluación v2 guardada en: C:\Users\elezr\Datalquemy\Proyectos\CNH\ml\evaluacion\eval_ayatsil_advanced_v2.json
